In [30]:
import cv2
import numpy as np

In [31]:
#Parameters for shi-tomasi corner detection
st_params = dict(maxCorners =30,
                qualityLevel = 0.2,
                minDistance = 2,
                blockSize = 7)

In [34]:
#Parmeters for Lucas-kande optical flow
lk_params = dict(winSize=(15,15),
                maxLevel =2,
                criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1))

In [35]:
#Video Capture
cap = cv2.VideoCapture('../Videos/pedestrian.mp4')

In [36]:
# 1. Color for optical flow
color = (0, 255,0)   # Green

# 2. Reading capture and get the first frame
ret, first_frame = cap.read()
first_frame = rescale_frame(first_frame, percent=25)

# 3. Conveet to grayscale
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

# 4. Strongest corner in the first frame
prev = cv2.goodFeaturesToTrack(prev_gray, mask=None, **st_params)

# 5. Create an image with same dimensions asthe frame for later drawing purpose
mask = np.zeros_like(first_frame)

In [37]:
def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [38]:
while (cap.isOpened()):
    ret, frame = cap.read()
    frame = rescale_frame(frame, percent=25)
    #convert all frames to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #calculate optical flow by Lucas-Kande
    next, status, error = cv2.calcOpticalFlowPyrLK(prev_gray,gray, prev, None, **lk_params)
    #Select good features for previous position
    good_old = prev[status == 1]
    # select good features for the next position
    good_new = next[status == 1]
    
    #Draw optical flow track
    for i , (new,old) in enumerate(zip(good_new, good_old)):
        # Return coordinates for new point
        a,b = new.ravel()
        # Return coordinates for the old points
        c,d = old.ravel()
        # Draw  line b/w new and old position
        mask = cv2.line(mask, (a,b), (c,d), color, 2)
        # Draw filled circle
        frame = cv2.circle(frame, (a,b), 3, color, -1)
    
    #Overlay optical flow on original frame
    
    output = cv2.add(frame, mask)
    # Update previous frame
    prev_gray = gray.copy()
    
    #Update previous good features
    prev = good_new.reshape(-1,1,2)
    
    #Open new window and display the output
    cv2.imshow("Optical Flow", output)
    if cv2.waitKey(300) & 0xFF == ord("q"):
        break
# Realese and Destroy
cap.release()
cv2.destroyAllWindows()